In [2]:
#Installing Libraries
import pandas as pd
import numpy as np
import math, copy
import matplotlib.pyplot as plt
import csv

In [3]:
#Uploading and visualizing the training dataset
train_df=pd.read_csv('train.csv')
train_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [4]:
train_df['TotalYears'] = train_df['YrSold'] - train_df['YearBuilt']
train_df['TotalYearsRemodel'] = train_df['YrSold'] - train_df['YearRemodAdd']
x_train= train_df[['LotFrontage','LotArea','YearBuilt', 'YearRemodAdd', 'TotalYears', 'TotalYearsRemodel', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF','GrLivArea', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces','GarageCars',
'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']]#assigning the input variables
x_train= x_train.to_numpy()

y_train=train_df['SalePrice'] #assigning the output variable
y_train=y_train.to_numpy()

x_train[np.isnan(x_train)] = 0


In [5]:
# data is stored in numpy array/matrix
print(f"x Shape: {x_train.shape}, X Type:{type(x_train)})")
print(x_train)
print(f"y Shape: {y_train.shape}, y Type:{type(y_train)})")
print(y_train)

x Shape: (1460, 27), X Type:<class 'numpy.ndarray'>)
[[   65.  8450.  2003. ...     0.     0.     0.]
 [   80.  9600.  1976. ...     0.     0.     0.]
 [   68. 11250.  2001. ...     0.     0.     0.]
 ...
 [   66.  9042.  1941. ...     0.     0.  2500.]
 [   68.  9717.  1950. ...     0.     0.     0.]
 [   75.  9937.  1965. ...     0.     0.     0.]]
y Shape: (1460,), y Type:<class 'numpy.ndarray'>)
[208500 181500 223500 ... 266500 142125 147500]


In [6]:
def zscore_normalize_features(x):
    """
    computes  X, zcore normalized by column
    
    Args:
      X (ndarray (m,n))     : input data, m examples, n features
      
    Returns:
      X_norm (ndarray (m,n)): input normalized by column
      mu (ndarray (n,))     : mean of each feature
      sigma (ndarray (n,))  : standard deviation of each feature
    """
    # find the mean of each column/feature
    mu     = np.mean(x, axis=0)                 # mu will have shape (n,)
    # find the standard deviation of each column/feature
    sigma  = np.std(x, axis=0)                  # sigma will have shape (n,)
    # element-wise, subtract mu for that column from each example, divide by std for that column
    x_norm = (x - mu) / sigma      

    return x_norm
 

In [7]:
x_norm = zscore_normalize_features (x_train)

In [8]:
n=x_norm.shape[1]
w_init= np.ones((n,))
b_init=20
print(f"x Shape: {x_norm.shape}, X Type:{type(x_norm)})")
print(x_train)
print(f"w Shape: {w_init.shape}, w Type:{type(w_init)})")
print(w_init)

x Shape: (1460, 27), X Type:<class 'numpy.ndarray'>)
[[   65.  8450.  2003. ...     0.     0.     0.]
 [   80.  9600.  1976. ...     0.     0.     0.]
 [   68. 11250.  2001. ...     0.     0.     0.]
 ...
 [   66.  9042.  1941. ...     0.     0.  2500.]
 [   68.  9717.  1950. ...     0.     0.     0.]
 [   75.  9937.  1965. ...     0.     0.     0.]]
w Shape: (27,), w Type:<class 'numpy.ndarray'>)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]


In [9]:
#defining the multiple linear regression model function, through parameters w and b
def model(x, w, b): #x -> input vector, with all features ; w and b -> parameters

    f = np.dot(x,w) + b 
        
    return f

In [10]:
def cost_function(x, y, w, b):
    m = x.shape[0]
    cost = 0.0
    for i in range(m):
        f = np.dot(x[i],w) + b          #(n,)(n,) = scalar (see np.dot)
        cost = cost + (f - y[i])**2       #scalar
    cost = cost / (2 * m)                      #scalar    
    return cost

In [11]:
cost = cost_function(x_norm, y_train, w_init, b_init)
print(f'Cost at initial w : {cost}')

Cost at initial w : 19515513916.507072


In [12]:
#Defining the gradient function for parameters w and b

def gradient_function(x,y,w,b):

    m, n = x.shape    
    dw = np.zeros((n,))
    db = 0
    for i in range(m):  
        err = (np.dot(x[i], w) + b) - y[i]   
        for j in range (n):
            dw[j] =dw[j] + err * x[i, j]  
            
        db= db + err
        
    dw = dw / m 
    db = db / m 
        
    return dw, db
    

In [13]:
tmp_dj_db, tmp_dj_dw = gradient_function(x_norm, y_train, w_init, b_init)
print(f'dj_dw at initial w,b: {tmp_dj_db}')
print(f'dj_db at initial w,b: \n {tmp_dj_dw}')

dj_dw at initial w,b: [-16644.23058045 -20949.57988911 -41523.95236419 -40269.18945896
  41559.92711305  40426.2389666  -30684.83625006    904.66227198
 -17030.49830445 -48721.93021774 -48107.77310776 -25356.58334781
   2034.89438292 -56268.13664909 -13354.67070524  10794.28210794
 -42379.3200487  -37076.71680786 -50853.20254302 -49504.67715266
 -25760.51895842 -25080.44825051  10211.48489426  -3539.5647625
  -8848.70670372  -7335.77267793   1683.78867528]
dj_db at initial w,b: 
 -180901.1958904111


In [14]:
#Defining the actual gradient descent, through updating the parameters w and b

def gradient_descent_function(x,y,initial_w,initial_b, alpha,iterations,cost_function,gradient_function):
    J_history = []
    p_history = []
    
    w = initial_w
    b = initial_b
    

    for i in range(iterations):
        
        dw, db = gradient_function(x,y,w,b)     

    
        b = b - alpha * db                            
        w = w - alpha * dw

        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append(cost_function(x,y,w,b))
        
        if i% math.ceil(iterations/10) == 0:
           print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")
          
            
    return w, b, J_history          

In [15]:
initial_w = np.zeros_like(w_init)
initial_b = 0.
# some gradient descent settings
iterations = 10000
alpha = 3.0e-2
# run gradient descent 
w_final, b_final, J_hist = gradient_descent_function(x_norm, y_train, initial_w, initial_b,alpha, iterations,
                                                    cost_function, gradient_function)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")
m,_ = x_norm.shape
for i in range(m):
    print(f"prediction: {np.dot(x_norm[i], w_final) + b_final:0.2f}, target value: {y_train[i]}")

Iteration    0: Cost 17808120973.06   
Iteration 1000: Cost 727893115.16   
Iteration 2000: Cost 727887984.70   
Iteration 3000: Cost 727887919.11   
Iteration 4000: Cost 727887872.18   
Iteration 5000: Cost 727887832.96   
Iteration 6000: Cost 727887800.16   
Iteration 7000: Cost 727887772.74   
Iteration 8000: Cost 727887749.81   
Iteration 9000: Cost 727887730.63   
b,w found by gradient descent: 180921.20,[ 2266.3809188   3080.42159457  6518.70176623  4933.29784896
 -6257.10168752 -4565.9648409   7595.67796537  -352.28265764
  1401.84833408  9179.22456633 10732.78734405 11378.99878947
  -484.18373019 17303.93428115 -9833.12014496 -8309.95244384
 11308.80222476  4849.04126901 11102.70644233  1260.45207498
  3671.57855258   446.35141561  1698.36541512   753.38236247
  3672.8601945  -2438.1662857    -78.46517134] 
prediction: 218127.49, target value: 208500
prediction: 185157.21, target value: 181500
prediction: 214882.09, target value: 223500
prediction: 181536.73, target value: 1400

In [17]:
test_df=pd.read_csv('test.csv')
test_df['TotalYears'] = test_df['YrSold'] - test_df['YearBuilt']
test_df['TotalYearsRemodel'] = test_df['YrSold'] - test_df['YearRemodAdd']
x_test= test_df[['LotFrontage','LotArea', 'TotalYears','YearBuilt', 'YearRemodAdd', 'TotalYearsRemodel', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF','GrLivArea', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces','GarageCars',
'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']]#assigning the input variables
x_test= x_test.to_numpy()
x_test[np.isnan(x_test)] = 0
x_norm_test = zscore_normalize_features(x_test)
y_pred=model(x_norm,w_final,b_final)
y_pred=pd.Series(y_pred)
sub=pd.DataFrame()
sub['Id']=test_df['Id']
sub['SalePrice'] = y_pred
sub.to_csv("submission.csv", index=False)